# Layered Model using `emg3d`

### Note regarding runtime

The following environment variables were set before starting Jupyter:
```
export OMP_NUM_THREADS=1
export MKL_NUM_THREADS=1
export OPENBLAS_NUM_THREADS=1
```
This ensures that our code runs only on one thread. CPU-time is therefore the same as walltime (or even a tiny fraction smaller).

In [1]:
import emg3d
import numpy as np
import xarray as xr
from datetime import datetime
%load_ext memory_profiler

## Load model and survey

In [2]:
ds = xr.load_dataset('../block_model_and_survey.nc', engine='h5netcdf')

### Extract required info

In [3]:
# Get Mesh
hx, hy, hz = ds.attrs['hx'], ds.attrs['hy'], ds.attrs['hz']
x0 = ds.attrs['x0']
mesh_orig = emg3d.TensorMesh([hx, hy, hz], x0=x0)

# Get Layered Model
resh, resv = ds.attrs['resh_bg'], ds.attrs['resv_bg']

# Create model instance
model_orig = emg3d.Model(mesh_orig, property_x=resh, property_z=resv, mapping='Resistivity')

# Get Survey
src = ds.attrs['src']
strength = ds.attrs['strength']
freq = ds.attrs['freq']
rec_x = ds.x.data[::2]
rec_y = ds.attrs['rec_y']
rec_z = ds.attrs['rec_z']

# Get source center (for meshing)
src_c = np.mean(src.reshape(-1, 2), 1)

## Computation mesh

In [4]:
mesh = emg3d.construct_mesh(
    frequency=freq,
    properties=[0.3, 1000, 100, 1000],
    center=[50, 0, 0],
    domain=[[-10050, 10050], [-4000, 4000], [-3150, 0]],
    min_width_limits=[100, 200, 50],
    max_buffer=50000,
    lambda_from_center=True,
)
mesh

TensorMesh: 1,572,864 cells

                      MESH EXTENT             CELL WIDTH      FACTOR
  dir    nC        min           max         min       max      max
  ---   ---  ---------------------------  ------------------  ------
   x    256    -57,087.78     65,201.21    100.00  8,113.42    1.17
   y     64    -51,967.77     51,967.77    200.00 14,623.87    1.43
   z     96    -35,514.71     65,482.02     50.00 19,403.13    1.42

In [5]:
# Interpolate to computational mesh
model = model_orig.interpolate2grid(mesh_orig, mesh, method='volume', log=True)

## `emg3d` computation

In [6]:
# Source field
sfield = emg3d.fields.get_source_field(mesh, src, freq, strength=strength)

In [7]:
mem = %memit -o efield, info = emg3d.solve(mesh, model, sfield, semicoarsening=23, linerelaxation=56, verb=3, return_info=True)
time = f"{info['time']:.0f} s"
ram = f"{(mem.mem_usage[0] - mem.baseline)/1024:.3f} GiB"
print(f"memory usage: {ram}")


:: emg3d START :: 10:55:53 :: v0.12.0

   MG-cycle       : 'F'                 sslsolver : False
   semicoarsening : True [2 3]          tol       : 1e-06
   linerelaxation : True [5 6]          maxit     : 50
   nu_{i,1,c,2}   : 0, 2, 1, 2          verb      : 3
   Original grid  : 256 x  64 x  96     => 1,572,864 cells
   Coarsest grid  :   2 x   2 x   3     => 12 cells
   Coarsest level :   7 ;   5 ;   5   

   [hh:mm:ss]  rel. error                  [abs. error, last/prev]   l s

       h_
      2h_ \                                                      /
      4h_  \                                        /\          / 
      8h_   \                            /\        /  \        /  
     16h_    \                  /\      /  \      /    \      /   
     32h_     \          /\    /  \    /    \    /      \    /    
     64h_      \    /\  /  \  /    \  /      \  /        \  /     
    128h_       \/\/  \/    \/      \/        \/          \/      

   [10:56:18]   1.871e-03  aft

In [8]:
# Extract (interpolate) Ex-field at receiver locations from the emg3d result.
egd = np.zeros((rec_x.size, 3), dtype=complex)
for i, y in enumerate(rec_y):
    egd[:, i] = emg3d.get_receiver(mesh, efield.fx, (rec_x, y, rec_z))

## Save data

In [9]:
# Save the three lines
ds.line_1.data = np.vstack([egd[:, 0].real, egd[:, 0].imag]).ravel('F')
ds.line_2.data = np.vstack([egd[:, 1].real, egd[:, 1].imag]).ravel('F')
ds.line_3.data = np.vstack([egd[:, 2].real, egd[:, 2].imag]).ravel('F')

# Add info
ds.attrs['runtime'] = time
ds.attrs['n_procs'] = 1
ds.attrs['max_ram'] = ram
ds.attrs['n_cells'] = f"({mesh.nCx} x {mesh.nCy} x {mesh.nCz}) - {mesh.nC}"
ds.attrs['n_nodes'] = 'N/A'
ds.attrs['n_dof'] = mesh.nE
ds.attrs['extent'] = (f"x = {mesh.vectorNx[0]:.1f}-{mesh.vectorNx[-1]:.1f}; " # Total mesh extent
                      f"y = {mesh.vectorNy[0]:.1f}-{mesh.vectorNy[-1]:.1f}; "
                      f"z = {mesh.vectorNz[0]:.1f}-{mesh.vectorNz[-1]:.1f}")
ds.attrs['min_vol'] = f"{np.min(mesh.vol):.1f}"
ds.attrs['max_vol'] = f"{np.max(mesh.vol):.1f}"
ds.attrs['machine'] = "Laptop; i7-6600U CPU@2.6 GHz x4; 15.5 GiB of memory, Ubuntu 18.04"
ds.attrs['version'] = f"emg3d v{emg3d.__version__}"
ds.attrs['date'] = datetime.today().isoformat()

# Save it under <{model}_{code}.nc>
ds.to_netcdf(f"../results/layered_emg3d.nc", engine='h5netcdf')

In [10]:
emg3d.Report()

--------------------------------------------------------------------------------
  Date: Fri Sep 04 10:57:50 2020 CEST

                OS : Linux
            CPU(s) : 4
           Machine : x86_64
      Architecture : 64bit
               RAM : 15.5 GB
       Environment : Jupyter

  Python 3.8.5 | packaged by conda-forge | (default, Aug 29 2020, 01:22:49)
  [GCC 7.5.0]

             numpy : 1.19.1
             scipy : 1.5.2
             numba : 0.51.2
             emg3d : 0.12.0
           empymod : 2.0.2
            xarray : 0.16.0
        discretize : 0.5.0
              h5py : 2.10.0
        matplotlib : 3.3.1
           IPython : 7.18.1
--------------------------------------------------------------------------------